# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Done
# Dependencies and Setup
# Takes a while to load
import hvplot.pandas
import pandas as pd
import requests
import datetime
import warnings
import holoviews as hv
from holoviews.operation.datashader import datashade

warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key
#I want to declare variables in the first cell
city_data_df = pd.read_csv("output_data/cities.csv")
print(f'Done importing: {datetime.datetime.now()}')

Done importing: 2024-04-01 13:56:11.871795


In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
print(f'Done importing: {datetime.datetime.now()}')
city_data_df.head(1)

Done importing: 2024-04-01 13:56:16.256118


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,praia da vitoria,38.7333,-27.0667,61.75,77,20,21.9,PT,1710544946


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
#I have tried to get the background map to display, including switching browsers, but nothing has worked.
#I also asked ChatGPT for help.  I tried about 6 re-writes but no worked.
#%%capture --no-display
import hvplot.pandas
import holoviews as hv
from holoviews.element import tiles

# Define the background image using OpenStreetMap tiles
background = tiles.OSM()

# Define the scatter plot
scatter_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', size='Humidity', 
                                           hover_cols=['City'], cmap='Blues', 
                                           title='City Humidity Map', 
                                           xlabel='Longitude', ylabel='Latitude', 
                                           colorbar=True, width=800, height=600)

# Overlay the background tiles with the scatter plot
map_plot = background * scatter_plot

# Display the map plot
map_plot

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
#Done
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[(city_data_df['Humidity'] >= 0) & (city_data_df['Humidity'] <= 40) & (city_data_df['Country'] == 'US')]
# & (city_data_df['Max Temp'] >= 90) & (city_data_df['Max Temp'] <= 120)

# Drop any rows with null values
filtered_df_clean = filtered_df.dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,lincoln city,44.9582,-124.0179,71.74,37,3,7.43,US,1710544814
208,208,laguna,38.4210,-121.4238,78.76,24,0,16.11,US,1710544801
230,230,placer,39.0666,-120.7677,65.52,25,0,5.01,US,1710545049
239,239,red hill,33.9996,-79.2581,68.92,1,100,11.83,US,1710545055
402,402,hutchinson,38.0608,-97.9298,61.74,40,0,5.75,US,1710544862


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
#Done
# Filter for only necessary items
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

#Hotel Name column added
hotel_df['Hotel Name'] = ""

hotel_df.head(1)

,City,Country,Lat,Lng,Humidity,Hotel Name
92,lincoln city,US,44.9582,-124.0179,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Hotel Search Parameters
radius = 10000
limit = 20

#Below is wrong
categories = "accommodation.hotel"
#conditions = "vegetarian"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Define filters and bias using latitude and longitude
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params = {
        "categories": categories,
        #"conditions": conditions,
        "limit": limit,
        "filter": filters,
        "bias": bias,
        "apiKey": geoapify_key    
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lincoln city - nearest hotel: Inn at Lincoln City
laguna - nearest hotel: Holiday Inn Express & Suites
placer - nearest hotel: Forest House Lodge
red hill - nearest hotel: No hotel found
hutchinson - nearest hotel: Fairfield Inn & Suites by Marriott
newman - nearest hotel: No hotel found
el granada - nearest hotel: Beach House
fergus falls - nearest hotel: AmericInn by Wyndham Fergus Falls
north bend - nearest hotel: The Mill Casino Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
92,lincoln city,US,44.9582,-124.0179,37,Inn at Lincoln City
208,laguna,US,38.4210,-121.4238,24,Holiday Inn Express & Suites
230,placer,US,39.0666,-120.7677,25,Forest House Lodge
239,red hill,US,33.9996,-79.2581,1,No hotel found
402,hutchinson,US,38.0608,-97.9298,40,Fairfield Inn & Suites by Marriott
430,newman,US,37.3138,-121.0208,38,No hotel found
484,el granada,US,37.5027,-122.4694,27,Beach House
507,fergus falls,US,46.2830,-96.0776,39,AmericInn by Wyndham Fergus Falls
528,north bend,US,43.4065,-124.2243,28,The Mill Casino Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
# Configure the map plot_4
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "City"
)

# Display the map plot
map_plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

In [49]:
import hvplot.pandas  # Ensure hvplot is imported
from bokeh.models import LabelSet, ColumnDataSource

# Convert DataFrame to ColumnDataSource
source = ColumnDataSource(hotel_df)

# Configure the map plot_4
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriNatGeo",
    frame_width=700,
    frame_height=500,
    scale=0.01,
    color="City"
)

# Add hotel names as labels to the plot
labels = LabelSet(
    x="Lng",
    y="Lat",
    text="Hotel Name",
    text_font_size='10pt',
    x_offset=5,
    y_offset=5,
    text_alpha=0.7,
    source=source
)

# Add labels to the plot
map_plot_4 = map_plot_4 * labels

# Display the map plot
map_plot_4


TypeError: unsupported operand type(s) for *: 'Overlay' and 'LabelSet'

In [51]:
import hvplot.pandas  # Ensure hvplot is imported
from bokeh.models import LabelSet, ColumnDataSource

# Convert DataFrame to ColumnDataSource
source = ColumnDataSource(hotel_df)

# Configure the map plot_4
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriNatGeo",
    frame_width=700,
    frame_height=500,
    scale=0.01,
    color="City"
)

# Add hotel names as labels to the plot
labels = LabelSet(
    x="Lng",
    y="Lat",
    text="Hotel Name",
    text_font_size='10pt',
    x_offset=5,
    y_offset=5,
    text_alpha=0.7,
    source=source
)

# Overlay the labels onto the plot
map_plot_4 = map_plot_4 * labels

# Display the map plot
map_plot_4


TypeError: unsupported operand type(s) for *: 'Overlay' and 'LabelSet'

In [54]:
import hvplot.pandas  # Ensure hvplot is imported
from bokeh.models import ColumnDataSource

# Convert DataFrame to ColumnDataSource
source = ColumnDataSource(hotel_df)

# Configure the map plot_4
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriNatGeo",
    frame_width=700,
    frame_height=500,
    scale=0.01,
    color="City"
)

# Add hotel names as labels to the plot
map_plot_4.text(
    x="Lng",
    y="Lat",
    text="Hotel Name",
    text_font_size='10pt',
    x_offset=5,
    y_offset=5,
    text_alpha=0.7,
    source=source
)

# Display the map plot
map_plot_4


AttributeError: 'Overlay' object has no attribute text.

In [56]:
import folium

# Create a Folium map centered around a mean point
map_plot_4 = folium.Map(location=[hotel_df['Lat'].mean(), hotel_df['Lng'].mean()], zoom_start=5, tiles='Stamen Terrain')

# Iterate through the DataFrame and add markers with hotel names to the map
for index, row in hotel_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Lng']],
        popup=row['Hotel Name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_plot_4)

# Display the map
map_plot_4
